In [1]:
from oa.model import ConditionNet

/home/xmcao/.conda/envs/install/envs/oa_reactdiff/lib/python3.10/site-packages/torch_geometric/typing.py:54: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.29' not found (required by /home/xmcao/.conda/envs/install/envs/oa_reactdiff/lib/python3.10/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/home/xmcao/.conda/envs/install/envs/oa_reactdiff/lib/python3.10/site-packages/torch_geometric/typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.29' not found (required by /home/xmcao/.conda/envs/install/envs/oa_reactdiff/lib/python3.10/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [2]:
model = ConditionNet()

In [7]:
def register(name, age):
	print(1) #SyntaxError：关键字参数name=‘Tom’在位置参数18之前
register(1, age=1)

1


In [8]:
import torch

In [9]:
torch.randn(10, 3) * torch.randn(10, 9)

RuntimeError: The size of tensor a (3) must match the size of tensor b (9) at non-singleton dimension 1

In [35]:
c=torch.tensor([[1],
              [0],
              [1]])

In [36]:
a = torch.randn(2, 3)
print(a)

tensor([[-0.1061,  0.9441, -0.8745],
        [ 0.5193,  0.2852,  1.9486]])


In [38]:
a[:, c.squeeze().bool()]

tensor([[-0.1061, -0.8745],
        [ 0.5193,  1.9486]])